In [1]:
import pandas as pd
from selenium import webdriver
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
from time import sleep
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
dir_for_shinhan = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'

In [3]:
region_data = pd.read_csv('C:/Users/shic/Desktop/crawler_project/data_save/kb_land/region_code.csv')

In [4]:
region_data['big_region_name'].unique()

array(['서울', '경기', '인천', '수도권', '부산', '대구', '광주', '대전', '울산', '5개광역시',
       '6개광역시', '세종', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주/서귀포',
       '기타지방'], dtype=object)

In [6]:
region_code = region_data[region_data['big_region_name'].isin(['서울','경기','인천'])]

In [7]:
driver = webdriver.Chrome(dir_for_shinhan)
driver.maximize_window()
driver.refresh()

In [8]:
kb_list = []
base_url = 'https://onland.kbstar.com/quics?page=C060737&%EB%B2%95%EC%A0%95%EB%8F%99%EB%8C%80%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C={}&%EB%B2%95%EC%A0%95%EB%8F%99%EC%A4%91%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C={}&%EA%B8%B0%EC%A4%80%EB%85%84={}&%EA%B8%B0%EC%A4%80%EC%9B%94={}&QSL=F'

for year in range(2015,2022):
    year = str(year)
    for month in range(1,13):
        if int(month) < 10:
            month = '0' + str(month)
        else:
            month = str(month)
        for region in region_code.iterrows():
            # format은 대지역 코드, 중지역 코드, 년, 월 순.
            tmp_url = base_url.format(region[1]['big_region_code'],region[1]['small_region_code'],year,month)
            
            # URL 접속
            driver.get(url = tmp_url) 
            
            # 그래프의 데이터 가져오기
            soup = BeautifulSoup(driver.page_source,'html.parser')
            
            aa = re.search('var graphYn1 = [$](.+?);',driver.page_source,re.S).group(1)
            aa = aa.replace(".trim('[","").replace("') == '' ? false: true","").replace('Date.UTC(','')
            aa = aa.split('], [')
            
            date_list_aa = []
            data_list_aa = []
            for k in aa:
                tmpp = k.split('),')
                data_wrangle = tmpp[0].split(', ')
                data_wrangle[1] = str(int(data_wrangle[1]) + 1)
                
                date_list_aa.append(','.join(data_wrangle))
                data_list_aa.append(float(tmpp[1].replace(']','')))
                
            bb = re.search('var graphYn2 = [$](.+?);',driver.page_source,re.S).group(1)
            bb = bb.replace(".trim('[","").replace("') == '' ? false: true","").replace('Date.UTC(','')
            bb = bb.split('], [')
            
            date_list_bb = []
            data_list_bb = []
            for k in bb:
                tmpp = k.split('),')
                data_wrangle = tmpp[0].split(', ')
                data_wrangle[1] = str(int(data_wrangle[1]) + 1)
                
                date_list_bb.append(','.join(data_wrangle))
                data_list_bb.append(float(tmpp[1].replace(']','').replace(',','')))
                
            kb_list.append(pd.DataFrame({
                'b_region' : region[1]['big_region_name'],
                's_region' : region[1]['small_region_name'],
                'date_a'   : pd.to_datetime(date_list_aa,format='%Y,%m,%d'),
                'value1'   : data_list_aa,
                'date_b'   : pd.to_datetime(date_list_bb,format='%Y,%m,%d'),
                'value2'   : data_list_bb
            }))
            
            # 일단 2021년 6월 까지의 데이터만 가져온다.
            if year == '2021' and int(month) > 6:
                break

driver.close()

In [9]:
df = pd.concat(kb_list,axis=0).drop_duplicates()
df['date_a'] = pd.to_datetime(df['date_a'],format='%Y-%m-%d')
df['date_b'] = pd.to_datetime(df['date_b'],format='%Y-%m-%d')

In [12]:
print(df['b_region'].unique())
print(df['s_region'].unique())

['서울' '경기' '인천']
['서울강북' '서울강북구' '서울광진구' '서울노원구' '서울도봉구' '서울동대문구' '서울마포구' '서울서대문구' '서울성동구'
 '서울성북구' '서울용산구' '서울은평구' '서울종로구' '서울중구' '서울중랑구' '서울강남' '서울강남구' '서울강동구'
 '서울강서구' '서울관악구' '서울구로구' '서울금천구' '서울동작구' '서울서초구' '서울송파구' '서울양천구' '서울영등포구'
 '수원' '수원장안구' '수원권선구' '수원팔달구' '수원영통구' '성남' '성남수정구' '성남중원구' '성남분당구' '고양'
 '고양덕양구' '고양일산동구' '고양일산서구' '안양' '안양만안구' '안양동안구' '부천' '의정부' '광명' '평택' '안산'
 '안산단원구' '안산상록구' '과천' '구리' '남양주' '용인' '용인처인구' '용인기흥구' '용인수지구' '시흥' '군포'
 '의왕' '하남' '오산' '파주' '이천' '안성' '김포' '양주' '동두천' '광주' '화성' '인천중구' '인천동구'
 '인천미추홀구' '인천연수구' '인천남동구' '인천부평구' '인천계양구' '인천서구']


In [14]:
#df.to_csv('C:/Users/shic/Desktop/crawler_project/data_save/kb_land/kb_land_df.csv',index=0)

In [2]:
tmp_df = pd.read_csv('C:/Users/shic/Desktop/crawler_project/data_save/kb_land/kb_land_df.csv')

In [3]:
tmp_df

,b_region,s_region,date_a,value1,date_b,value2
0,서울,서울강북,2014-03-03,77.3,2014-03-03,0.05
1,서울,서울강북,2014-03-10,77.3,2014-03-10,0.04
2,서울,서울강북,2014-03-17,77.3,2014-03-17,0.05
3,서울,서울강북,2014-03-24,77.3,2014-03-24,0.03
4,서울,서울강북,2014-03-31,77.4,2014-03-31,0.02
...,...,...,...,...,...,...
35691,경기,김포,2021-07-05,134.2,2021-07-05,0.15
35692,경기,양주,2021-07-05,117.3,2021-07-05,0.46
35693,경기,동두천,2021-07-05,120.9,2021-07-05,0.54
35694,경기,광주,2021-07-05,125.8,2021-07-05,0.67
